# Users

This notebook contains examples of using the ArcGIS API for Python to manage users in an ArcGIS Online portal. It assumes you are relatively familiar with Python generally, and with performing ArcGIS Online organization administration tasks through the portal web app. It does not assume any familiarity with the specifics of how to use the Python API to automate those administrative tasks.

## Table of Contents

* [Notebook Environment](#notebook-environment)
* [Create `User` objects](#create-user-objects)
    * [List all users in an organization](#list-users)
    * [Search for users](#search-users)
    * [Get a specific user](#get-user)
    * [Get the user associated with a portal connection](#get-the-user-associated-with-a-portal-connection)
* [Get information about a user](#get-info)
    * [Display user information](#display-info)
    * [Access user properties](#access-properties)
    * [Find items owned by a user](#find-items)
* [Manage users](#manage)
    * [Create a single user](#create-single)
    * [Create multiple users](#create-multiple)
    * [Update a user's information](#update)
    * [Reset a user's password](#reset)
    * [Update a user's license type](#update-license)
    * [Update a user's role](#update-role)
    * [Allocate credits to a user](#allocate)
    * [Reassign a user's content](#reassign)
    * [Delete a single user](#delete-single)
    * [Delete multiple users](#delete-multiple)




## Notebook Environment

The code in this notebook is specific to version 2.2 of the Python API. All API references in this notebook are from the [arcgis.gis](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html) module: [CreditManager](https://developers.arcgis.com/python/api-reference/arcgis.gis.admin.html#creditmanager), [ContentManager](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#contentmanager), [GIS](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#gis), [User](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#user), and [UserManager](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#usermanager)

The code below imports the relevant portions of the `arcgis` package, confirms you are using the correct version, and creates a `GIS` object called `gis` that connects you to your ArcGIS Online portal. The variables in this cell are required for subsequent cells to work, but the cells in this notebook are otherwise independent of each other. 

You will need to alter the code creating the `GIS` object to make a connection to your portal. Various methods of creating this object are covered in the [Portal connection notebook](https://github.com/travisormsby/agol_admin/blob/main/portal_connection.ipynb). Usernames and specific content details (such as item id) used in this notebook are also specific to my organization. You will need to change them to something appropriate for your organization.

In [1]:
import arcgis
from arcgis.gis import GIS

assert arcgis.__version__.startswith("2.2.")
gis = GIS(profile="agol") # Change to make a connection to your portal

## Create `User` objects <a id='create-user-objects'></a>

In order to manage users, you will need to represent them as some data structure that Python can manipulate. An instance of the [`UserManager` class](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#usermanager) can create these `User` objects. This instance is available as the `users` property of a [`GIS` object](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#gis)

### List all users in an organization <a id='list-users'></a>

The [`org_search` method of the UserManager object](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.UserManager.org_search) will generate a tuple of `User` objects. By default, it will return all users in your organization. <a id='org_search'></a>

In [246]:
all_users = gis.users.org_search()
for user in all_users:
    print(user)

<User username:ormsby_guest1>
<User username:t.ormsby>
<User username:ormsby_guest2>


### Search for users <a id='search-users'></a>

You can also use `org_search` to find users in your organization that match a query and sort the `User` objects returned.

In [247]:
some_users = gis.users.org_search(query="ormsby_guest", sort_field="username", sort_order="desc")
for user in some_users:
    print(user)

<User username:ormsby_guest2>
<User username:ormsby_guest1>


In addition to `org_search`, you can use the [`search`](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.UserManager.search) and [`advanced_search`](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.UserManager.advanced_search) methods to search for users. These methods can search across ArcGIS Online.<a id='search'></a>

### Get a specific user <a id='get-user'></a>

The [`get` method of the UserManager object](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.UserManager.get)  will return a single `User` object that matches the provided username. The `get` method is not limited to a specific organization and can return any ArcGIS Online user. <a id='get'></a>

In [248]:
one_user = gis.users.get(username="ormsby_guest1")
print(one_user)

<User username:ormsby_guest1>


### Get the user associated with a portal connection

Because it is possible to make many portal connections, each as a different user, it can be helpful to know what user is associated with a particular connection. The code below uses the [`me` property of the the `UserManager` object](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.UserManager.me) to retrieve this `User` object.

In [264]:
current_user = gis.users.me
print(current_user)

<User username:t.ormsby>


## Get information about a user <a id='get-info'></a>

Once you have a `User` object, you can access information about that user. 

### Display user information <a id='display-info'></a>

The code below uses the Jupyter Notebook `display` function to show a card for a user, including a link to their profile page.

In [249]:
user = gis.users.get("ormsby_guest1")
display(user)

<User username:ormsby_guest1>

### Access user properties <a id='access-properties'></a>

You can also access specific properties of a `User` object. The code below prints some example information you can access. The [API Reference for the `User` object](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#user) has the complete list. <a id='properties'></a>

In [250]:
user = gis.users.get("ormsby_guest1")
print(user.fullName)
print(user.availableCredits)
print(user.role)
print(user.groups)

Ormsby Guest1
10.0
org_publisher
[]


Most objects in Python have a `__dict__` property that shows all the object's properties. The code below leverages the `__dict__` property to represent the `User` object as a Python dictionary.

In [251]:
user = gis.users.get("ormsby_guest1")
user.__dict__

{'_gis': GIS @ https://ormsby.maps.arcgis.com version:2023.3,
 '_portal': <arcgis.gis._impl._portalpy.Portal at 0x7fda1dfc06a0>,
 '_user_id': 'ormsby_guest1',
 'thumbnail': None,
 '_workdir': '/tmp',
 '_invitemgr': None,
 '_hydrated': False,
 'username': 'ormsby_guest1',
 'udn': None,
 'id': '6626b8616d794725978b22fafdbdcc44',
 'fullName': 'Ormsby Guest1',
 'availableCredits': 10.0,
 'assignedCredits': 10.0,
 'categories': [],
 'emailStatus': 'notverified',
 'firstName': 'Ormsby',
 'lastName': 'Guest1',
 'preferredView': None,
 'description': None,
 'email': 't.ormsby@esri.com',
 'userType': 'arcgisonly',
 'idpUsername': None,
 'favGroupId': 'cfb0adf777ca48a383ae610543ab9356',
 'lastLogin': 1698009281000,
 'mfaEnabled': False,
 'mfaEnforcementExempt': False,
 'storageUsage': 445780599,
 'storageQuota': 2199023255552,
 'orgId': 'jgEF2qh8RkmonUq2',
 'role': 'org_publisher',
 'privileges': ['features:user:edit',
  'portal:publisher:bulkPublishFromDataStores',
  'portal:publisher:createDat

### Find items owned by a user <a id='find-items'></a>


<a id='item_search'></a> There is an [`items` method of a `User` object](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.User.items) but it can only be used to check items in a specified folder. 

For that reason, the code below uses the [`search` method of the `ContentManager` object](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.ContentManager.search) to generate a list of `Item` objects owned by a given username.

In [252]:
gis.content.search(query="owner:ormsby_guest1", max_items=20)

[<Item title:"ExampleMap3" type:Web Map owner:ormsby_guest1>,
 <Item title:"ExampleMap2" type:Web Map owner:ormsby_guest1>,
 <Item title:"ExampleMap1" type:Web Map owner:ormsby_guest1>]

## Manage users <a id='manage'></a>

With a `User` object, you can programatically manage a user. The following examples give a good idea of the types of things you can do with users, but are not comprehensive. The API Reference for the [User](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#user) and [UserManager](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#user) classes provides additional details.

### Create a single user

<a id='create-single'></a>The code below uses the the [`create` method of the `UserManager` object](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.UserManager.create) to make a new Viewer account in the built-in identity store. Specifying a password of `None` requires the user to set their password when they first attempt to log in to the portal.

Note that ArcGIS Online usernames that use the built-in identity store must be globally unique across all ArcGIS Online users. 

In [253]:
new_user = gis.users.create(
        username="fabb44dc",
        password=None,
        firstname="Test",
        lastname="User1",
        email="testuser1@example.com",
        role="iAAAAAAAAAAAAAAA", # Viewer role id
        user_type="viewerUT",
        credits=0
    )
display(new_user)

<User username:fabb44dc>

### Create multiple users <a id='create'></a>

<a id='create-multiple'></a>Because the `create` method only creates a single user, you will need to use a loop to create multiple users at a time. The code below uses a list of dictionaries of user values to create two new accounts. In this case, the collection of user values is hard coded as a dictionary, but you might read those data from a file instead.

In [254]:
user_data = [
    {"uname": "cedd2845", "fname": "Test", "lname": "User2", "email": "testuser2@example.com"},
    {"uname": "e23fb959", "fname": "Test", "lname": "User3", "email": "testuser3@example.com"}
]

for user in user_data:
    new_user = gis.users.create(
        username=user["uname"],
        password=None,
        firstname=user["fname"],
        lastname=user["lname"],
        email=user["email"],
        role="iAAAAAAAAAAAAAAA",
        user_type="viewerUT",
        credits=0
    )
    display(new_user)

<User username:cedd2845>

<User username:e23fb959>

### Update a user's information <a id='update'></a>

The code below updates a user's name, email, and description, using the [`update` method of the `User` object](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.User.update). The documentation has a comprehensive list of properties that can be changed with this method. <a id='update'></a>

In [255]:
user = gis.users.get("fabb44dc")
user.update(
    first_name="New First Name",
    last_name="New Last Name",
    email="newname@example.com",
    description="I feel like a brand new person"   
)
display(user)


<User username:fabb44dc>

### Reset a user's password <a id='reset'></a>

Accounts that use IT-managed identity stores like Active Directory or SAML cannot be managed with the ArcGIS API for Python. But an account that uses the built-in identity store can be. The code below uses the [`reset` method of a `User` object](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.User.reset) to send a user an email with a temporary password and instructions to reset it the next time they log in.

In [256]:
user = gis.users.get("fabb44dc")
user.reset(password=None, reset_by_email=True)

True

### Update a user's license type <a id='update-license'></a>

A user's license type governs what types of roles and privileges they can be granted. In this example, the user has a Viewer license type. The code below changes that to a Creator license type using the [`update_license_type` method of a `User` object](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.User.update_license_type).

In [257]:
user = gis.users.get("fabb44dc")
user.update_license_type("CreatorUT")

True

### Update a user's role <a id='update-role'></a>

A user's role governs the privileges they have in the portal, and must be compatible with their license type. In this example, the user has a Viewer role. The code below changes that to a Publisher role using the [`update_role` method of the `User` object](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.User.update_role)

In [258]:
user = gis.users.get("fabb44dc")
user.update_role("org_publisher")

True

### Allocate credits to a user <a id='allocate'></a>

The code below uses the [`allocate` method of the `CreditManager` class in the admin submodule](https://developers.arcgis.com/python/api-reference/arcgis.gis.admin.html#arcgis.gis.admin.CreditManager.allocate) to allocate credits to a user based on their username.

In [259]:
gis.admin.credits.allocate(username="fabb44dc", credits=10)

True

### Reassign a user's content <a id='reassign'></a>

The code below reassigns all of a user's items and groups to a different user with the [`reassign_to` method of a `User` object](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.User.reassign_to). This operation can only be performed by an administrator, occurs immediately, and cannot be undone.

This operation recreates the original folder structure of items in the new owner's content.

In [260]:
user1 = gis.users.get("ormsby_guest1")
user2 = gis.users.get("fabb44dc")
user1.reassign_to(target_username=user2)

True

The code below uses the [`transfer_content` method of a `User` object](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.User.transfer_content) to transfer the content of a folder to another user. This asyncronous operation may take up to 15 minutes to complete. <a id='transfer_content'></a>

This operation transfers all items to a single folder in the new owner's content, regardless of the original folder structure.

In [261]:
user1 = gis.users.get("ormsby_guest1")
user2 = gis.users.get("fabb44dc")
user2.transfer_content(target_user=user1)

<Future at 0x7fda1dfd1a80 state=finished returned dict>

### Delete a single user <a id='delete-single'></a>

The code below uses the [`delete` method of a `User` object](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.User.delete) to delete that user. Users that own content or groups will need to have them reassigned to another user.

In [262]:
user = gis.users.get("fabb44dc")
user.delete()

True

### Delete multiple users <a id='delete-multiple'></a>

There is a [`delete_users` method of the `UserManager` object](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.UserManager.delete_users) that accepts a list of `User` objects. This operation, however, does not actually delete the users, it merely removes them from the organization. You cannot see them when searching for users, but you can still access their `User` object with the `get` method, and you cannot create new users with the same username as these removed users.  

For that reason, the code below deletes multiple users by looping through users and using the `delete` method. This strategy is slower than the `delete_users` method, but more effective in permanently deleting users.

In [263]:
unames = ["cedd2845","e23fb959"]
for uname in unames:
    user = gis.users.get(uname)
    user.delete()